In [12]:
!pip install ultralytics

In [2]:
pip install transformers accelerate pillow

Sau khi cài đặt thư viện, chúng ta sẽ tải mô hình và bộ xử lý từ thư mục `blip2-opt-2.7b`. Hãy đảm bảo rằng thư mục này đã chứa các tệp mô hình và cấu hình cần thiết.

In [8]:
from transformers import AutoProcessor, AutoModelForImageTextToText
import torch

# Đường dẫn đến thư mục chứa mô hình của bạn
model_path = './models/blip2-opt-2.7b'

# Tải bộ xử lý và mô hình
processor = AutoProcessor.from_pretrained(model_path)
model = AutoModelForImageTextToText.from_pretrained(model_path)
device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)

print("Mô hình và bộ xử lý đã được tải thành công!")

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Mô hình và bộ xử lý đã được tải thành công!


In [25]:
from transformers import AutoProcessor, AutoModelForVision2Seq
from PIL import Image
import torch

def describe_frame_with_prompt(frame_np, prompt, vlm_processor, vlm_model):
    raw_image = Image.fromarray(frame_np.astype("uint8"))

    inputs = vlm_processor(
        images=raw_image,
        text=prompt,
        return_tensors="pt"
    )

    inputs = {k: v.to("cuda") for k, v in inputs.items()}

    with torch.no_grad():
        # Thêm max_new_tokens để đảm bảo câu trả lời không bị cắt ngắn
        out = vlm_model.generate(**inputs, max_new_tokens=100) # Có thể điều chỉnh giá trị này

    caption = vlm_processor.decode(out[0], skip_special_tokens=True)

    # Loại bỏ tiền tố 'Question: ... Answer:' nếu có
    if "Answer: " in caption:
        caption = caption.split("Answer: ", 1)[1].strip()
    elif "question:" in caption.lower() and "answer:" in caption.lower():
        # Xử lý trường hợp chữ hoa/thường khác nhau
        answer_start_index = caption.lower().find("answer:")
        if answer_start_index != -1:
            caption = caption[answer_start_index + len("answer:"):].strip()

    return caption

Nhận Diện và trích xuất Frames

In [33]:
import os
from ultralytics import YOLO
from modules.extract_frames import extract_frames_to_queue
from modules.tracker import BestFrameTracker
from utils.SaveFrame import save_track_frame

model_path_yolo = "models/best.pt" # Đường dẫn cho mô hình YOLO
video_path = r"train/videos/03cde2e3_322_clip_017_0123_0129_N.mp4"

yolo_detector = YOLO(model_path_yolo) # Đổi tên biến mô hình YOLO
frames_queue = extract_frames_to_queue(video_path)
tracker = BestFrameTracker()

frame_count = 0
while True:
    frame = frames_queue.get()
    if frame is None:
        break

    frame_count += 1

    # Object detection và tracking
    results = yolo_detector.track(frame, tracker="bytetrack.yaml", verbose=False)

    if not results or len(results) == 0:
        continue

    # Xử lý từng object được detect
    for box in results[0].boxes:
        if box.id is None:  # Bỏ qua nếu không có track ID
            continue

        # Lấy thông tin box
        x1, y1, x2, y2 = box.xyxy[0].cpu().numpy().astype(int)
        track_id = int(box.id)
        confidence = float(box.conf)
        cls_id = int(box.cls)
        cls_name = results[0].names[cls_id] if hasattr(results[0], "names") else str(cls_id)

        # Cập nhật tracker
        bbox = (x1, y1, x2, y2)
        tracker.update_track(frame, track_id, bbox, confidence, cls_name)


all_caption = []
# Sau khi tracking, chúng ta sẽ điền vào list frames toàn cục
for track_id, frameData in tracker.best_frames.items():
    box = frameData.box_info
    vlm_instruction_prompt = f"Question: Describe the surrounding environment and context of the car. Furthermore, what is the location of the traffic sign associated with the bounding box {box.bbox}? Answer:"
    caption_from_vlm = describe_frame_with_prompt(frameData.frame, vlm_instruction_prompt, processor, model) # Sử dụng VLM model và processor toàn cục

    all_caption.append(f"Caption Frame {track_id}: {caption_from_vlm} Information the traffic sign:[label: '{box.class_name}', score: '{frameData.score}']")
print(all_caption)

🎞️ Xử lý video 03cde2e3_322_clip_017_0123_0129_N.mp4 | FPS gốc: 30.0
✅ Trích xuất frame hoàn tất: 03cde2e3_322_clip_017_0123_0129_N.mp4
["Caption Frame 1: The car is driving on a highway in the countryside, with a few trees and a few houses in the background. The sky is cloudy, with a few clouds in the sky. The road is a two-lane road with a few trees on the left side. The sign is on the right side of the road. Information the traffic sign:[label: 'IE.DISTANCE: Khoảng cách', score: '0.8010389788406372']"]
